#  T1 and T2 times

In [1]:
import matplotlib.pyplot as plt
from equal1.benchmarking.device_characterization import thermal_noise
import pandas as pd
import seaborn as sns
from tqdm import tqdm

In [2]:
hardware = {
    "hpc-conservative": 32,
    "hpc-gamma1": 32,
    "hpc-gamma2": 32,
}

In [3]:
all_experiments_t1 = [
    thermal_noise.IdleNoiseT1(device_name=device, qubit_idx=qubit)
    for device, num_qubits in hardware.items()
    for qubit in range(num_qubits)
]

all_experiments_t2 = [
    thermal_noise.ThermalNoiseT2Ramsey(device_name=device, qubit_idx=qubit)
    for device, num_qubits in hardware.items()
    for qubit in range(num_qubits)
]

In [ ]:
results = []

for t1_experiment, t2_experiment in tqdm(zip(all_experiments_t1, all_experiments_t2)):
    assert t1_experiment.device_name == t2_experiment.device_name
    assert t1_experiment.qubit_idx == t2_experiment.qubit_idx

    device_name = t1_experiment.device_name
    qubit_idx = t1_experiment.qubit_idx

    t1_experiment.run(
        simulate=True,
        runtime_options={
            "backend": "TensorNet_MPS",
            "simulation_platform": "GPU",
        },
    )
    t1 = t1_experiment.analyse_results()

    t2_experiment.run(
        simulate=True,
        runtime_options={
            "backend": "TensorNet_MPS",
            "simulation_platform": "GPU",
        },
    )
    t2 = t2_experiment.analyse_results()

    results.append(
        {
            "device": device_name,
            "qubit_idx": qubit_idx,
            "t1": t1,
            "t2": t2,
        }
    )

0it [00:00, ?it/s]/home/ninjaforest/equal1/equal1-benchmarking/.venv/lib/python3.13/site-packages/qbraid/runtime/native/provider.py:155: RuntimeWarning: The default runtime configuration for device 'equal1_simulator' includes transpilation to program type 'cudaq', which is not registered.
  warnings.warn(
0it [00:26, ?it/s]


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(results)
df.to_csv("t1_t2_results.csv", index=False)

In [ ]:
fig, ax = plt.subplots(figsize=(24, 6))
sns.barplot(data=df, x="qubit_idx", y="t1", hue="device", ax=ax)
ax.set_title("T1 Time by Qubit and Device")
ax.set_ylabel("T1 Time (s)")
ax.set_xlabel("Qubit Index")
ax.legend(title="Device")

In [ ]:
fig, ax = plt.subplots(figsize=(24, 6))
sns.barplot(data=df, x="qubit_idx", y="t2", hue="device", ax=ax)
ax.set_title("T2 Time by Qubit and Device")
ax.set_ylabel("T2 Time (s)")
ax.set_xlabel("Qubit Index")
ax.legend(title="Device")